In [1]:
import numpy as np
import pandas as pd
import torch
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
import sys
import numpy as np
from numpy.random import seed
### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow as tf


import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Input, add
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import load_model, save_model, Model


from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *


tf.__version__

'2.3.0'

In [3]:

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
# seed(random_seed)
# tf.random.set_seed(random_seed)
# np.random.seed(random_seed)



In [4]:
def seed_everything(seed= 1000):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed= random_seed)  

In [5]:
def acti_model(input_shape):
        
        model = Sequential()
        
        model.add(Conv1D(10, 44, strides=2,activation='relu', input_shape=input_shape))
        model.add(Conv1D(10, 20, strides=2, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
        model.add(Conv1D(10, 4, strides=2, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        return model


In [6]:
from collections import defaultdict
from data_loader import loadEvents
meal_ls = []
def get_meal_info(person_name = None,file_ls = [], file_ls_doc=None,root_path = "../data/",print_file=False,round_decimal=1):
        """
        if file_ls is not given, then get file_ls according to person_name
        file path = root_path + file name in all_files_list.txt

        return:
            meal event count, total minutes of all meals, total hours of all meals,total day counts

        """
        meal_counts = 0
        sec_counts = 0
        min_counts = 0
        hour_counts = 0
        total_hours = 0
        total_mins = 0
        total_sec = 0
        day_counts = 0 
        if person_name ==None:
            return meal_counts, min_counts,hour_counts, day_counts, total_hours

        data_indices_file = "../data-file-indices/" +person_name+"/all_files_list.txt"
        fp = open(data_indices_file,"r")
        txt = fp.read()
        fp.close()
        file_ls = txt.split("\n")
        while '' in file_ls:
            file_ls.remove('')
       
        day_counts = len(file_ls)
        
        for i, file_name in enumerate(file_ls):
            print()
            print()
            print("File: ",file_name)
            file_name = root_path + file_name
            TotalEvents, EventStart, EventEnd, EventNames, TimeOffset,EndTime = loadEvents(file_name, debug_flag = False, print_file=print_file)
            meal_counts += TotalEvents
            total_sec +=  abs(EndTime - TimeOffset)
            for i in range(len(EventStart)):
                t_diff = (EventEnd[i]- EventStart[i])//15
                if t_diff >0:
                    print( "===> Meal len (second):",t_diff, t_diff/3600.)
                    sec_counts += ( EventEnd[i]- EventStart[i])//15
                    meal_ls.append(t_diff/3600.)
        print()
        print()
#         print("Total sec count: ", sec_counts,sec_counts/3600)
        print()
        
        total_hours = total_sec/(60*60)
        min_counts = sec_counts/60
        hour_counts = min_counts/60
        average_meal_per_day = meal_counts/len(file_ls)
        average_hour_per_meal = hour_counts/meal_counts
        # round numbers
        total_hours = round(total_hours, round_decimal)
        min_counts = round(min_counts, round_decimal)
        hour_counts = round(hour_counts, round_decimal)
        average_meal_per_day = round(average_meal_per_day,round_decimal)
        average_hour_per_meal = round(average_hour_per_meal, round_decimal)
        
        no_eating_hours = total_hours - hour_counts
        weight_ratio = round(no_eating_hours/hour_counts, round_decimal)
        result = pd.DataFrame({"dataset": person_name,"Days":day_counts, 
                      "Total Hours":total_hours,"Meal Counts":meal_counts,
                      "Average Meal Counts Per Day":average_meal_per_day,"Average Hours Per Meal": average_hour_per_meal,
                      "Eating Hours":hour_counts, "No Eating Hours":no_eating_hours,
                     "Balance Ratio(no_eat/eat)":weight_ratio},index=[0])
    
        return result

          
def get_dataset_info(names= ["wenkanw"],winmin=6,stridesec=5):
    meal_info = defaultdict(list)
    dataset_results = pd.DataFrame()
    for name in names:
        result = get_meal_info(person_name=name)
        if dataset_results.empty:
            dataset_results = result
        else:
            dataset_results = dataset_results.append(result,ignore_index=True)
    
    # append total summary
#     print( dataset_results)
    total_result=pd.DataFrame({"dataset":"total"},columns = dataset_results.columns,index=[0])
    # append average summary
    average_result=pd.DataFrame({"dataset":"average"},columns = dataset_results.columns,index=[0])
    key_ls = ["Days","Total Hours","Meal Counts","Eating Hours","No Eating Hours"]
    for key in dataset_results.columns:
        if key in key_ls:
            total_result[key].at[0] = round(dataset_results[key].sum() ,1)
            average_result[key].at[0] = round(dataset_results[key].mean(),1)

    ls = [total_result, average_result]
    for df in ls:
        df["Average Meal Counts Per Day"].at[0] = round(df["Meal Counts"].values[0]/df["Days"].values[0], 1)
        df["Average Hours Per Meal"].at[0] =round( df["Eating Hours"].values[0]/df["Meal Counts"].values[0], 1)
        df["Balance Ratio(no_eat/eat)"].at[0] =round(df["No Eating Hours"].values[0]/df["Eating Hours"].values[0],1)
        dataset_results =dataset_results.append(df,ignore_index=True)

    return dataset_results

In [9]:
# name_ls = ["lawler"]
name_ls = ["tianyi"]
# name_ls = ["wenkanw","adam","lawler","shaurya"]
dataset_info = get_dataset_info(names= name_ls)
dataset_info



File:  IndividualData/tianyi-data/1-18-21/Shimmer-Data.shm
===> Meal len (second): 1620 0.45


File:  IndividualData/tianyi-data/2-3-21/Shimmer-Data.shm
===> Meal len (second): 1080 0.3
===> Meal len (second): 1080 0.3


File:  IndividualData/tianyi-data/1-25-21/Shimmer-Data.shm
===> Meal len (second): 1200 0.3333333333333333
===> Meal len (second): 600 0.16666666666666666


File:  IndividualData/tianyi-data/1-19-21/Shimmer-Data.shm
===> Meal len (second): 1500 0.4166666666666667


File:  IndividualData/tianyi-data/1-26-21/Shimmer-Data.shm
===> Meal len (second): 780 0.21666666666666667


File:  IndividualData/tianyi-data/1-20-21/Shimmer-Data.shm
===> Meal len (second): 1380 0.38333333333333336


File:  IndividualData/tianyi-data/1-31-21/Shimmer-Data.shm
===> Meal len (second): 300 0.08333333333333333
===> Meal len (second): 1200 0.3333333333333333


File:  IndividualData/tianyi-data/1-16-21/Shimmer-Data.shm
===> Meal len (second): 1680 0.4666666666666667
===> Meal len (second): 900 

,dataset,Days,Total Hours,Meal Counts,Average Meal Counts Per Day,Average Hours Per Meal,Eating Hours,No Eating Hours,Balance Ratio(no_eat/eat)
0,tianyi,14,127.8,20,1.4,0.3,5.9,121.9,20.7
1,total,14,127.8,20,1.4,0.3,5.9,121.9,20.7
2,average,14,127.8,20,1.4,0.3,5.9,121.9,20.7


In [53]:
pd.read_csv("../results/dataset_info.csv")

,dataset,Days,Total Hours,Meal Counts,Average Meal Counts Per Day,Average Hours Per Meal,Eating Hours,No Eating Hours,Balance Ratio(no_eat/eat)
0,wenkanw,17.0,129.2,32.0,1.9,0.2,7.7,121.5,15.8
1,adam,14.0,125.6,26.0,1.9,0.1,3.3,122.3,37.1
2,lawler,23.0,173.9,61.0,2.7,0.4,25.9,148.0,5.7
3,shaurya,13.0,134.0,38.0,2.9,0.2,9.3,124.7,13.4
4,total,67.0,562.7,157.0,2.3,0.3,46.2,516.5,11.2
5,average,16.8,140.7,39.2,2.3,0.3,11.6,129.1,11.1


In [46]:
! cat ../data/IndividualData/lawler-data/10.26/10.26-events.txt

START	2020-10-25	19:27:20
Lunch	19:55	20:01	bedroom	Seconds	Alone	CompanyNotEating	bowl of BBQ pulled pork | watching a show on PC
Snack	20:05	20:08	bedroom	Seconds	Alone	CompanyNotEating	poptart pack  | using PC
Dinner	23:52	00:00	bedroom	Seconds	Alone	CompanyNotEating	Wendys biggie bag | using PC
END	2020-10-26	01:09:02






In [8]:
!cat ../data/IndividualData/james-data/1-25-2021/Day4-events.txt

START	2021-01-25	11:48:49
Breakfast	12:09:00	12:28:00	Home	NoSeconds	InCompany	CompanyNotEating	bagel, Granola PB wrap, protein shake | TV
Dinner	19:15:00	19:47:00	Home	Seconds	InCompany	CompanyNotEating	Burrito | TV
END	2021-01-25	23:37:48





In [10]:
! cat ../data/IndividualData/tianyi-data/1-14-21/Shimmer-Data-events.txt

START	2021-01-14	15:42:41
Dinner	20:48	21:40	Home	NoSeconds	Alone	CompanyNotEating	Rice Noodle, Coke Zero | TV, Talking
END	2021-01-14	23:49:41





In [31]:
meal_ls.sort()
meal_ls

[0.008333333333333333,
 0.013333333333333334,
 0.027777777777777776,
 0.03333333333333333,
 0.03333333333333333,
 0.03333333333333333,
 0.03333333333333333,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.08333333333333333,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.11666666666666667,
 0.11666666666666667,
 0.11666666666666667,
 0.13333333333333333,
 0.13333333333333333,
 0.13333333333333333,
 0.15,
 0.15,
 0.15,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.17972222222222223,
 0.2,
 0.21666666666666667,
 0.25,
 0.3333333333333333,
 0.3333333333333333,
 0.4166666666666667,
 0.55,
 0.75,
 18.305]

In [1]:
from packages import *
print(os.path.isfile("."))

False


In [3]:
def f():
    return np.array([1,2,3])
f()

array([1, 2, 3])

In [12]:
a = np.array(["ab","ac","bc"])
b= np.array(["ab","ac"])
np.where(a in b)

ValueError: shape mismatch: objects cannot be broadcast to a single shape